# Geometry analysis

## MDTrajを使ってgeometry analysis

Geometry関連の関数は[ドキュメント](https://www.mdtraj.org/1.9.7/analysis.html)から確認することができます。ここではいくつかを紹介します。

In [ ]:
import mdtraj as md
import numpy as np

traj = md.load('../md/protein.dcd', top='../md/protein.pdb')

原子間距離を計算します

In [ ]:
import matplotlib.pyplot as plt

dist = md.compute_distances(traj, [[0, 100], [0, 200]])

# Plot inter-domain distance
plt.figure(figsize=(12, 5))
plt.plot(dist)
plt.xlabel('Frame')
plt.ylabel('atom-pair distance [nm]')
plt.legend(['atom 0-100', 'atom 0-200'])
plt.show()

ドメイン間の距離を計算します

In [ ]:
domain2_indices = traj.topology.select('resSeq 82 to 164')
print(domain2_indices)

In [ ]:
traj_domain2 = traj.atom_slice(domain2_indices)
print(traj_domain2)

In [ ]:
md.compute_center_of_mass(traj_domain2).shape

In [ ]:
import matplotlib.pyplot as plt

# Define two domains using atom selections
domain1_indices = traj.topology.select('residue 1 to 60')
domain2_indices = traj.topology.select('resSeq 82 to 164')

# Compute centers of mass
domain1_com = md.compute_center_of_mass(traj.atom_slice(domain1_indices))
domain2_com = md.compute_center_of_mass(traj.atom_slice(domain2_indices))

# Calculate distance between centers of mass
com_distance = np.sqrt(np.sum((domain1_com - domain2_com)**2, axis=1))

# Plot inter-domain distance
plt.figure(figsize=(12, 5))
plt.plot(com_distance)
plt.xlabel('Frame')
plt.ylabel('Inter-domain distance [nm]')
plt.show()

トラジェクトリの一部における残基間のコンタクトの頻度を計算します

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Calculate the contact distances for each frame
distances, residue_pairs = md.compute_contacts(traj[:10], scheme='closest-heavy')

# Average over all frames to get the mean contact map
mean_distances = distances.mean(axis=0)

# Create an empty 2D array for the contact map
distance_map = np.zeros((traj.topology.n_residues, traj.topology.n_residues))

# Fill the contact map with the mean contacts
for (i, j), distance in zip(residue_pairs, mean_distances):
    distance_map[i, j] = distance
    distance_map[j, i] = distance

# Plot the contact map
plt.figure(figsize=(8, 8))
plt.imshow(distance_map, cmap='viridis', origin='lower')
plt.colorbar(label='Mean contact frequency')
plt.xlabel('Residue index')
plt.ylabel('Residue index')
plt.show()